In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.tools.plotting import scatter_matrix
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import metrics
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn import neighbors, tree, naive_bayes
from sklearn.lda import LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import decomposition
from datetime import datetime

In [3]:
#load data
water_train = pd.read_csv("C:\Datasets\water_train.csv")#, index_col=0)
water_test = pd.read_csv("C:\Datasets\water_test.csv")#, index_col=0)
water_records_trans2 = pd.read_csv("C:\Datasets\water_records_trans2.csv")#, index_col=0)
watertest_records_trans2 = pd.read_csv("C:\Datasets\watertest_records_trans2.csv")#, index_col=0)
water_target = pd.read_csv("C:\Datasets\water_training_labels.csv")
water_target = water_target.drop('id', 1)

In [9]:
#class distribution
#functional: 32259/59400 (54.308%)
#non-functional: 22824/59400 (38.424%)
#func needs repair: 4317/59400 (7.268%)

In [29]:
water_train_ssf=pd.get_dummies(water_train)
water_test_ssf=pd.get_dummies(water_test)

In [12]:
water_target_array=np.array(water_target)
water_target_array=water_target_array.squeeze()
water_target_array

array(['functional', 'functional', 'functional', ..., 'functional',
       'functional', 'functional'], dtype=object)

In [ ]:
#Model Construction

In [13]:
water_train2, water_test2, water_target_train, water_target_test = train_test_split(water_train_ssf, water_target_array, test_size=0.34)
print water_train2.shape, water_test2.shape, water_target_train.shape, water_target_test.shape 
#water_train2, water_test2, water_target_train, water_target_test = train_test_split(water_records_trans2, water_target, test_size=0.34)
#print water_train2.shape, water_test2.shape, water_target_train.shape, water_target_test.shape 


(39204, 225) (20196, 225) (39204L,) (20196L,)


In [14]:
water_train2.head(5)

,amount_tsh,gps_height,longitude,latitude,population,days_frm_20131213,Month_Recorded,funder_0,funder_Danida,funder_Dhv,...,Construction_Year_Str_2004,Construction_Year_Str_2005,Construction_Year_Str_2006,Construction_Year_Str_2007,Construction_Year_Str_2008,Construction_Year_Str_2009,Construction_Year_Str_2010,Construction_Year_Str_2011,Construction_Year_Str_2012,Construction_Year_Str_Other
35799,0,0,31.768048,-3.028071,0,876,7,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14050,1000,331,37.528171,-6.436614,90,1017,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
38958,0,0,31.923172,-3.477988,0,436,10,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27802,0,2199,31.846868,-8.108498,500,840,8,0,0,0,...,0,0,0,0,0,0,0,0,0,1
41099,0,0,32.611978,-2.596000,0,880,7,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [15]:
water_target_train_np=np.array(water_target_train)
water_target_train_np=np.squeeze(water_target_train_np)
water_target_train_np.shape

(39204L,)

In [16]:
water_target_train_np

array(['functional', 'functional', 'functional', ..., 'non functional',
       'functional', 'functional'], dtype=object)

In [17]:
water_target_test_np=np.array(water_target_test)
water_target_test_np=np.squeeze(water_target_test_np)
water_target_test_np.shape

(20196L,)

In [18]:
#random forest
#Grid search using Random Forest
rf = RandomForestClassifier()
#rf.get_params()

In [19]:
#    'criterion': ['entropy', 'gini'],

rfparameters = {
    'n_estimators': range(500, 501, 5)
}
#    'max_depth': np.linspace(1, 20, 10),
#    'min_samples_split': np.linspace(2, 20, 10)
#}

rf_gs = GridSearchCV(rf, rfparameters, verbose=1, cv=10)

In [20]:
#%time _ = rf_gs.fit(water_ssf, water_target_np)
#rf_gs.best_params_, rf_gs.best_score_

In [21]:
rf = RandomForestClassifier(criterion="entropy", n_estimators=500)
#criterion='entropy',max_depth=3,min_samples_split=6, 
rf.fit(water_train2, water_target_train_np)
rf_predictions=rf.predict(water_train2)
print "Accuracy:{0:.5f}".format(metrics.accuracy_score(water_target_train_np, rf_predictions)),"\n"
print "Classification report"
print metrics.classification_report(water_target_train_np, rf_predictions),"\n"
print "Confussion matrix"
print metrics.confusion_matrix(water_target_train_np, rf_predictions),"\n"

Accuracy:0.99885 

Classification report
             precision    recall  f1-score   support

 functional       1.00      1.00      1.00     21268
functional needs repair       1.00      0.99      0.99      2814
non functional       1.00      1.00      1.00     15122

avg / total       1.00      1.00      1.00     39204


Confussion matrix
[[21257     9     2]
 [   23  2789     2]
 [    7     2 15113]] 



In [22]:
rf = RandomForestClassifier(criterion='entropy', n_estimators=500)
#,max_depth=3,min_samples_split=6, 
rf.fit(water_train2, water_target_train_np)
rf_predictions=rf.predict(water_test2)
print "Accuracy:{0:.5f}".format(metrics.accuracy_score(water_target_test_np, rf_predictions)),"\n"
print "Classification report"
print metrics.classification_report(water_target_test_np, rf_predictions),"\n"
print "Confussion matrix"
print metrics.confusion_matrix(water_target_test_np, rf_predictions),"\n"

Accuracy:0.80120 

Classification report
             precision    recall  f1-score   support

 functional       0.81      0.88      0.84     10991
functional needs repair       0.53      0.35      0.42      1503
non functional       0.83      0.78      0.80      7702

avg / total       0.79      0.80      0.80     20196


Confussion matrix
[[9637  333 1021]
 [ 743  520  240]
 [1543  135 6024]] 



In [30]:
rf = RandomForestClassifier(criterion='entropy', n_estimators=500)
#,max_depth=3,min_samples_split=6, 
rf.fit(water_train_ssf, water_target_array)
rf_predictions=rf.predict(water_test_ssf)

In [34]:
rf_predictions[:5]

array(['functional', 'functional', 'functional', 'non functional',
       'functional'], dtype=object)

In [39]:
rf_predictions_pd=pd.DataFrame(rf_predictions)

In [38]:
water_test_full = pd.read_csv("C:\Datasets\water_test_data.csv")#, index_col=0)
water_test_id = water_test_full['id']
water_test_id.head(5)

0    50785
1    51630
2    17168
3    45559
4    49871
Name: id, dtype: int64

In [52]:
water_test_predictions=pd.concat([water_test_id, rf_predictions_pd],axis=1)
water_test_predictions[:5]

,id,0
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [53]:
water_test_predictions.rename(columns={0: 'status_group'}, inplace=True)

In [54]:
water_test_predictions[:5]

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional


In [56]:
water_test_predictions.to_csv("water_test_predictions.csv", index=False)